In [1]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()


In [25]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [26]:
soout_col_names = ['SO_INDEX', 'SHT_ROW_IDX', 'PERSON_IN_CHARGE', 'SHIP_DATE', 'DM_KEY', 'POD_KEY', 'IS_LOCAL','POD_DATE', 'TIMELINE']
len(soout_col_names)

9

In [64]:
df_soout = pd.DataFrame(cursor.execute("select * from so_out").fetchall())
df_soout.columns = soout_col_names
df_soout = df_soout.sort_values('SO_INDEX')
df_soout.set_index('SO_INDEX',inplace=True)
df_soout

,SHT_ROW_IDX,PERSON_IN_CHARGE,SHIP_DATE,DM_KEY,POD_KEY,IS_LOCAL,POD_DATE,TIMELINE
SO_INDEX,,,,,,,,
1,si_20475,하하,2023-03-01,DM_1,POD_1,no_local,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
6,si_20474,하하,2023-03-01,DM_1,POD_1,no_local,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
7,si_20444C20446,에이티지,2023-03-01,DM_1,POD_1,lc_46C49,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
11,si_20449,에케테스트,2023-03-05,DM_1,POD_1,lc_1471,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
12,si_20455,하하,2023-03-01,DM_1,POD_1,lc_1469,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."
13,only_local,온리로컬,2023-03-01,DM_1,POD_1,lc_1468,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."


### POD xl sheet 만들기

In [41]:
tmp_si_idx = df_soout['SHT_ROW_IDX'][0]
tmp_lc_idx = df_soout['IS_LOCAL'][0]

'si_20474'

In [65]:
df_soout

SHT_ROW_IDX                                            si_20444C20446
PERSON_IN_CHARGE                                                 에이티지
SHIP_DATE                                                  2023-03-01
DM_KEY                                                           DM_1
POD_KEY                                                         POD_1
IS_LOCAL                                                     lc_46C49
POD_DATE                                                         None
TIMELINE            {"meta": {"desc": "data_update_timeline", "col...
Name: 7, dtype: object

In [44]:
tmp_si_idx = df_soout['SHT_ROW_IDX'][0]
tmp_lc_idx = df_soout['IS_LOCAL'][0]

In [47]:
si_and_lc_list = [get_each_index_num(tmp_si_idx), get_each_index_num(tmp_lc_idx)]
si_and_lc_list

['20474', 'local']

In [124]:
df_soout['SO_DETAIL'] = None
df_soout['LC_DETAIL'] = None


for so_index in range(len(df_soout)):
    tmp_si_idx = df_soout['SHT_ROW_IDX'].iloc[so_index]
    tmp_lc_idx = df_soout['IS_LOCAL'].iloc[so_index]
    si_and_lc_list = [get_each_index_num(tmp_si_idx), get_each_index_num(tmp_lc_idx)]
    df_so = pd.DataFrame()
    df_lc = pd.DataFrame()
    for i,val in enumerate(si_and_lc_list):

        if val == 'local':
            None
        else:
            if type(val) is list:
                idx_list = val
            elif type(val) is dict:
                idx_list=val['idx_list']
            else:
                idx_list = [val]
            for idx in idx_list:
                if i == 0:
                    db_obj = cursor.execute(f"SELECT * FROM SHIPMENT_INFORMATION WHERE SI_INDEX = '{idx}'").fetchone()
                    df_db_obj = pd.DataFrame([db_obj])
                    df_so = pd.concat([df_so,df_db_obj])
                else:
                    db_obj = cursor.execute(f"SELECT * FROM LOCAL_LIST WHERE LC_INDEX = '{idx}'").fetchone()
                    df_db_obj = pd.DataFrame([db_obj])
                    df_lc = pd.concat([df_lc,df_db_obj])
                if not df_so.empty:
                    df_soout['SO_DETAIL'].iloc[so_index] = df_so
                if not df_lc.empty:
                    df_soout['LC_DETAIL'].iloc[so_index] = df_lc


In [125]:
df_soout

,SHT_ROW_IDX,PERSON_IN_CHARGE,SHIP_DATE,DM_KEY,POD_KEY,IS_LOCAL,POD_DATE,TIMELINE,SO_DETAIL,LC_DETAIL,ORDER_NO,SHIP_TO
SO_INDEX,,,,,,,,,,,,
1,si_20475,하하,2023-03-01,DM_1,POD_1,no_local,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",0 1 2 3 4 ...,None,None,None
6,si_20474,하하,2023-03-01,DM_1,POD_1,no_local,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",0 1 2 3 4 ...,None,None,None
7,si_20444C20446,에이티지,2023-03-01,DM_1,POD_1,lc_46C49,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",0 1 2 3 4 ...,0 1 2 3 ...,None,None
11,si_20449,에케테스트,2023-03-05,DM_1,POD_1,lc_1471,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",0 1 2 3 4 ...,0 1 2 ...,None,None
12,si_20455,하하,2023-03-01,DM_1,POD_1,lc_1469,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",0 1 2 3 4 5...,0 1 2 ...,None,None
13,only_local,온리로컬,2023-03-01,DM_1,POD_1,lc_1468,None,"{""meta"": {""desc"": ""data_update_timeline"", ""col...",None,0 1 2 ...,None,None


In [128]:
df_soout['LC_DETAIL'].iloc[0] == None

True

In [123]:
df_soout['ORDER_NO'] = None
df_soout['SHIP_TO'] = None
for so_index in range(len(df_soout)):
    order_list = []
    shipto_list = []
    if df_soout['SO_DETAIL'].iloc[so_index] != None:
    
    elif 
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,20475,STO0165597,115394024,140138466,1,2022-10-24,2122133,12999.0,None,바이넥스 오송공장,2022-10-31,2023-03-01,None,None,BJH_Binex_EP1-D-20887/8_CaptoS/Q,UPD3351866,None,SHIP_CONFIRM,"{""meta"": {""desc"": ""data_update_timeline"", ""col..."


In [109]:
# ORDER NM_LIST
order_list = []
list(df_soout['SO_DETAIL'].iloc[0][6])
df_soout['LC_DETAIL'].iloc[0]

In [111]:
# SHIP TO LIST
list(df_soout['SO_DETAIL'].iloc[0][9])

['바이넥스 오송공장']

In [61]:
def get_each_index_num(get_idx_str):
    """
    DW테이블 SO_OUT상의 si_index 및 is_local 컬럼값을 넣으면 해당 row의 고유 키값을 dict형태로 반환

    example : get_idx_str='si_13384A14171A14243C14244'  ==>
    {'out_sht_id':'si','idx_list':[13384, 14171, 14243, 14244]}
    """
    del_sht_id = get_idx_str.split('_')[0]
    get_idx_str = get_idx_str.split('_')[1]
    count_A = get_idx_str.count("A")
    count_C = get_idx_str.count("C")
    if count_A == 0 and count_C == 0:
        return get_idx_str
    procs_1 = get_idx_str.split("A")
    procs_1
    A_list = []
    C_list = []
    for val in procs_1:
        if val.count("C") > 0 :
            C_list.append(val)
        else :
            A_list.append(int(val))
    
    for c_val in C_list:      
        tmp_c = c_val.split('C')
        tmp_diff = int(tmp_c[1])-int(tmp_c[0])
        fin_C_list = []
        for i, val in enumerate(range(tmp_diff+1)):
            fin_C_list.append(int(tmp_c[0]) + i)
        A_list = A_list+fin_C_list
            
    return {'out_sht_id':del_sht_id,'idx_list':A_list}